In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from keras.utils import np_utils
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import cv2
from keras.layers import LeakyReLU
from keras import regularizers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, AveragePooling2D, Activation
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/fer2013/fer2013.csv")

In [ ]:
df.head()

In [ ]:
df['Usage'].unique()

In [ ]:
train = df[df['Usage']=='Training']
PublicTest = df[df['Usage']=='PublicTest']
PrivateTest = df[df['Usage']== 'PrivateTest']
test = pd.concat([PublicTest,PrivateTest])
print(len(train))

number of train samples and test samples same as data used in other notebook 

In [ ]:
train.emotion.value_counts()

In [ ]:
test.emotion.value_counts()

In [ ]:

pixels = df['pixels']
#print(pixels)
X = np.zeros((len(pixels), 48*48)) #images 48X48
for idx_x in range(X.shape[0]):

    vals = pixels[idx_x].split(' ')
    #print(len(vals)/48)
 
    for idx_y in range(X.shape[1]):
        X[idx_x,idx_y] = int(vals[idx_y])
    
        
X = X/255 #normalizing images

In [ ]:
im = X[0].copy()
im = im.reshape((48,48))
plt.imshow(m)

In [ ]:
Y = df['emotion']
Y = np_utils.to_categorical(Y, 7) 


In [ ]:
index = len(train)
X_train = X[0:index, :]
Y_train = Y[0:index]
X_crossval = X[index:, :]
Y_crossval = Y[index:]

print (X_train.shape, Y_train.shape)
print (X_crossval.shape, Y_crossval.shape)

In [ ]:
X_train = X_train.reshape((X_train.shape[0], 48, 48 ,1))
X_crossval = X_crossval.reshape((X_crossval.shape[0], 48, 48,1))

Data Augmentation Same as other Notebook

In [ ]:
datagen = ImageDataGenerator( zoom_range=0.3, horizontal_flip=True)

datagen.fit(X_train)

In [ ]:
    model = tf.keras.models.Sequential()   

    model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.1), input_shape=(48,48,1)))
    model.add(Conv2D(128, kernel_size=(3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.1)) #try 0.15

    model.add(Conv2D(256, kernel_size=(3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding='same'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))
    
    model.add(Conv2D(512, kernel_size=(3, 3), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.1), input_shape=(48,48,1)))
    model.add(Conv2D(512, kernel_size=(3, 3), activation= tf.keras.layers.LeakyReLU(alpha=0.1), padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.1))
    

    model.add(Flatten())

    model.add(Dense(512,kernel_regularizer=regularizers.l2(0.01),activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
    #above we get out 
    model.add(Dropout(0.2))

    model.add(Dense(7, activation='softmax'))

    model.compile(optimizer=Adam(lr=0.0001, decay=1e-6), 
              loss="categorical_crossentropy", metrics=['accuracy'])

    model.summary()

In [ ]:
batch_size = 64
epochs = 50
steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_crossval) // batch_size

In [ ]:
check_point_loss = tf.keras.callbacks.ModelCheckpoint(filepath='model_loss.h5',
                                                      save_best_only=True,
                                                      verbose=1,
                                                      mode='min',
                                                      moniter='val_loss')

check_point_acc = tf.keras.callbacks.ModelCheckpoint(filepath='model_acc.h5',
                                                      save_weights_only=True,
                                                      verbose=1,
                                                      mode='max',
                                                      moniter='val_accuracy')

# Reduce learning rate when a metric has stopped improving.
reduce_learning_rate = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                            factor=0.2, 
                                                            patience=5, 
                                                            verbose=1, 
                                                            min_delta=0.0001)


logs = tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1),
csv_logger = tf.keras.callbacks.CSVLogger('training.log')

callbacks = [check_point_loss,check_point_acc, reduce_learning_rate, csv_logger]

hist = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=64),
                           steps_per_epoch= steps_per_epoch,
                           epochs=50, 
                           verbose=1, 
                           validation_data=(X_crossval, Y_crossval), 
                           callbacks=callbacks)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from keras.models import Model
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier  
from sklearn import metrics
from sklearn import svm

In [ ]:
y_pred = model.predict_classes(X_crossval)
y_pred


In [ ]:
y_actual = df['emotion'][index:]
y_actual = np.asarray(y_actual)
y_actual 

In [ ]:
metrics.accuracy_score(y_actual  ,y_pred)

**Following Code for plotting a confusion matrix from:**

https://www.kaggle.com/grfiv4/plot-a-confusion-matrix

In [ ]:
import numpy as np


def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
cm =confusion_matrix(y_actual, y_pred)
cm

In [ ]:
labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
plot_confusion_matrix(cm = cm, normalize= False,target_names = labels,title = "Confusion Matrix for CNN")

**CNN + Another Classifier**

In [ ]:
model_feat = Model(model.input,outputs=model.layers[-3].output)
x_1 = model_feat.predict(X_train)
y_1 = np.asarray(df['emotion'][0:index])
x_2 = model_feat.predict(X_crossval)
y_2 = np.asarray(df['emotion'][index:])

One Hot Encoding:
Following Code for One Hot Encoding from https://www.analyticsvidhya.com/blog/2020/03/one-hot-encoding-vs-label-encoding-using-scikit-learn/

In [ ]:
onehotencoder1 = OneHotEncoder()
#reshape the 1-D country array to 2-D as fit_transform expects 2-D and finally fit the object 
Y1 = onehotencoder1.fit_transform(y_1.reshape(-1,1)).toarray()
onehotencoder2 = OneHotEncoder()
Y2 = onehotencoder1.fit_transform(y_2.reshape(-1,1)).toarray()

RFC1: One Hot Encoded

Accuracy = 0.5662533091821095

Accuracy very low so did not use one hot encoding for the rest of the RF  models

In [ ]:
rfc1 = RandomForestClassifier(random_state=42, n_estimators=300)
rfc1.fit(x_1, Y1)

In [ ]:
predrfc = rfc1.predict(x_2)
metrics.accuracy_score(predrfc ,Y2)

**RFC1: Without One Hot Encoding** 

Accuracy: 0.6968092517765083

In [ ]:
rfc1 = RandomForestClassifier(random_state=42, n_estimators=300)
rfc1.fit(x_1, y_1)

predrfc = rfc1.predict(x_2)
metrics.accuracy_score(predrfc ,y_2)

**Using Default RFC**

Accuracy:0.6961125818587154

In [ ]:
rfc2 = RandomForestClassifier()
rfc2.fit(x_1, y_1)

In [ ]:
predrfc2 = rfc2.predict(x_2)
metrics.accuracy_score(predrfc2 ,y_2)

RFC parameters 3

Accuracy:0.6969485857600669

In [ ]:
rfc3 = RandomForestClassifier(random_state=42, n_estimators=700)
rfc3.fit(x_1, y_1)

predrfc3 = rfc3.predict(x_2)
metrics.accuracy_score(predrfc3 ,y_2)

RFC parameters 4:

Accuracy:0.6937439041382193


In [ ]:
rfc4 = RandomForestClassifier(random_state=42, n_estimators=1200)
rfc4.fit(x_1, y_1)

predrfc4 = rfc4.predict(x_2)
metrics.accuracy_score(predrfc4 ,y_2)

XGB classifier
Accuracy 0.6910965584506061

In [ ]:
from xgboost import XGBClassifier
xgb1 =  XGBClassifier(n_estimators = 700,learning_rate=0.01)
xgb1.fit(x_1, y_1)
y_pred1 = xgb1.predict(x_2)
metrics.accuracy_score(y_2, y_pred1)

SVM rbf Kernal

Accuracy: 0.6862198690260555 (OVR)

Accuracy : 0.6912358924341647 (OVO)

In [ ]:
rbf_svc = svm.SVC(kernel='rbf',decision_function_shape='ovo')
rbf_svc.fit(x_1, y_1)

In [ ]:
pred_rbf_svc = rbf_svc.predict(x_2)
metrics.accuracy_score(pred_rbf_svc ,y_2)

SVM Linear Kernal

Accuracy: 0.6742371464400168

In [ ]:
linear_svc = svm.SVC(kernel='linear',decision_function_shape='ovo')
linear_svc.fit(x_1, y_1)

In [ ]:
pred_linear_svc = linear_svc.predict(x_2)
metrics.accuracy_score(pred_linear_svc ,y_2)

SVM Linear Kernal (ovr):

Accuracy: 0.6637870976731225


In [ ]:
linear_svc1 = svm.SVC(kernel='linear',decision_function_shape='ovr')
linear_svc1.fit(x_1, y_1)

In [ ]:
pred_linear_svc1 = linear_svc1.predict(x_2)
metrics.accuracy_score(pred_linear_svc1 ,y_2)